#### Pyspark Handling Missing Values
- Dropping Columns
- Dropping Rows
- Various Parameter In Dropping Functionalities
- Handling Missing Values

In [17]:
from pyspark.sql import SparkSession

In [18]:
spark=SparkSession.builder.appName('test').getOrCreate()

In [19]:
df_pyspark=spark.read.csv('test_data_tutorial3.csv', header=True, inferSchema=True)
df_pyspark.show()

+------+----+----+------+
|  name| age| exp|salary|
+------+----+----+------+
|   bob|  23|   7| 63000|
| saget|  26|   9| 75000|
| james|  21|   5| 25000|
|  paul|  29|   9|100000|
| rohan|  32|  10| 50000|
| jagat|  23|   2| 23000|
|mahesh|NULL|NULL| 40000|
|  NULL|  38|  12|110000|
|  NULL|  36|NULL|  NULL|
+------+----+----+------+



In [20]:
# drop rows with null values in them
df_pyspark.na.drop().show()

+-----+---+---+------+
| name|age|exp|salary|
+-----+---+---+------+
|  bob| 23|  7| 63000|
|saget| 26|  9| 75000|
|james| 21|  5| 25000|
| paul| 29|  9|100000|
|rohan| 32| 10| 50000|
|jagat| 23|  2| 23000|
+-----+---+---+------+



In [21]:
df_pyspark.na.drop(how="all").show() #drop a row if all the values are null
df_pyspark.na.drop(how="any").show() #default: drop if any of the values in the row are null
df_pyspark.na.drop(how="any", thresh=2).show() # drop a row if it doesn't have atleast 'thresh' NON-null values
df_pyspark.na.drop(how="any", subset=['exp']) # drop entire row if the specified column has a NAN value

+------+----+----+------+
|  name| age| exp|salary|
+------+----+----+------+
|   bob|  23|   7| 63000|
| saget|  26|   9| 75000|
| james|  21|   5| 25000|
|  paul|  29|   9|100000|
| rohan|  32|  10| 50000|
| jagat|  23|   2| 23000|
|mahesh|NULL|NULL| 40000|
|  NULL|  38|  12|110000|
|  NULL|  36|NULL|  NULL|
+------+----+----+------+

+-----+---+---+------+
| name|age|exp|salary|
+-----+---+---+------+
|  bob| 23|  7| 63000|
|saget| 26|  9| 75000|
|james| 21|  5| 25000|
| paul| 29|  9|100000|
|rohan| 32| 10| 50000|
|jagat| 23|  2| 23000|
+-----+---+---+------+

+------+----+----+------+
|  name| age| exp|salary|
+------+----+----+------+
|   bob|  23|   7| 63000|
| saget|  26|   9| 75000|
| james|  21|   5| 25000|
|  paul|  29|   9|100000|
| rohan|  32|  10| 50000|
| jagat|  23|   2| 23000|
|mahesh|NULL|NULL| 40000|
|  NULL|  38|  12|110000|
+------+----+----+------+



DataFrame[name: string, age: int, exp: int, salary: int]

In [23]:
# Filling the missing value
df_pyspark.na.fill('Missing Values').show() # repalce NANs with specifed value (only if the data-type matches). For instance, this statement will only replace missing (NULL) names.
df_pyspark.na.fill(10, ['age', 'exp']).show() # replace for specified columns

+--------------+----+----+------+
|          name| age| exp|salary|
+--------------+----+----+------+
|           bob|  23|   7| 63000|
|         saget|  26|   9| 75000|
|         james|  21|   5| 25000|
|          paul|  29|   9|100000|
|         rohan|  32|  10| 50000|
|         jagat|  23|   2| 23000|
|        mahesh|NULL|NULL| 40000|
|Missing Values|  38|  12|110000|
|Missing Values|  36|NULL|  NULL|
+--------------+----+----+------+

+------+---+---+------+
|  name|age|exp|salary|
+------+---+---+------+
|   bob| 23|  7| 63000|
| saget| 26|  9| 75000|
| james| 21|  5| 25000|
|  paul| 29|  9|100000|
| rohan| 32| 10| 50000|
| jagat| 23|  2| 23000|
|mahesh| 10| 10| 40000|
|  NULL| 38| 12|110000|
|  NULL| 36| 10|  NULL|
+------+---+---+------+



In [24]:
df_pyspark.show()

+------+----+----+------+
|  name| age| exp|salary|
+------+----+----+------+
|   bob|  23|   7| 63000|
| saget|  26|   9| 75000|
| james|  21|   5| 25000|
|  paul|  29|   9|100000|
| rohan|  32|  10| 50000|
| jagat|  23|   2| 23000|
|mahesh|NULL|NULL| 40000|
|  NULL|  38|  12|110000|
|  NULL|  36|NULL|  NULL|
+------+----+----+------+



In [25]:
# Replace the NULLs using imputer function

from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['age', 'exp', 'salary'],
    outputCols=["{}_imputed".format(c) for c in ['age', 'exp', 'salary']]
).setStrategy("mean")

In [26]:
imputer.fit(df_pyspark).transform(df_pyspark).show()

+------+----+----+------+-----------+-----------+--------------+
|  name| age| exp|salary|age_imputed|exp_imputed|salary_imputed|
+------+----+----+------+-----------+-----------+--------------+
|   bob|  23|   7| 63000|         23|          7|         63000|
| saget|  26|   9| 75000|         26|          9|         75000|
| james|  21|   5| 25000|         21|          5|         25000|
|  paul|  29|   9|100000|         29|          9|        100000|
| rohan|  32|  10| 50000|         32|         10|         50000|
| jagat|  23|   2| 23000|         23|          2|         23000|
|mahesh|NULL|NULL| 40000|         28|          7|         40000|
|  NULL|  38|  12|110000|         38|         12|        110000|
|  NULL|  36|NULL|  NULL|         36|          7|         60750|
+------+----+----+------+-----------+-----------+--------------+

